In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.preprocessing.image import ImageDataGenerator
import os
from keras.models import Sequential, Model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.advanced_activations import ELU
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import regularizers
from keras.regularizers import l1, l2
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.
/home/KTH/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/KTH/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/KTH/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/KTH/.local/lib/python3.6/site-packages/tensorflow/python/framework/d

In [2]:
num_classes = 7
img_rows, img_cols = 48, 48
batch_size = 32
epochs = 200

train_data_dir = '../fer2013/train'
validation_data_dir = '../fer2013/validation'

val_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 30,
    shear_range = 0.3,
    zoom_range = 0.3,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [3]:
print(validation_generator.class_indices)

{'Angry': 0, 'Disgust': 1, 'Fear': 2, 'Happy': 3, 'Neutral': 4, 'Sad': 5, 'Surprise': 6}


In [4]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 180:
        lr * 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [5]:
def resnet_layer(inputs,
                num_filters=16,
                kernel_size=3,
                strides=1,
                activation='relu',
                batch_normalization=True,
                conv_first=True):
    conv = Conv2D(num_filters,
                 kernel_size=strides,
                 padding='same',
                 kernel_initializer='he_normal',
                 kernel_regularizer=l2(1e-4))
    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [6]:
def resnet_v2(input_shape, depth, num_classes=7):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
    input_shape (tensor): shape of input image tensor
    depth (int): number of core convolutional layers
    num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
    model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
        
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)
    
    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs,
                    num_filters=num_filters_in,
                    conv_first=True)
    
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            # num of param setting 
            if stage == 0: # first stage
                num_filters_out = num_filters_in * 4
                if res_block == 0: # first layer & first stage
                    activation = None
                    batch_normalization = False
            else: # second, third stage
                num_filters_out = num_filters_in * 2
                if res_block == 0: # first layer but no first stage
                    strides = 2 # downsample
            y = resnet_layer(inputs=x,
                            num_filters=num_filters_in,
                            kernel_size=1,
                            strides=strides,
                            activation=activation,
                            batch_normalization=batch_normalization,
                            conv_first=False)
            y = resnet_layer(inputs=y,
                            num_filters=num_filters_in,
                            conv_first=False)
            y = resnet_layer(inputs=y,
                            num_filters=num_filters_out,
                            kernel_size=1,
                            conv_first=False)
            if res_block == 0:
                # Linear projection residual shortcut connection to match
                # changed dims
                # at the first time, make a shortcut origin
                x = resnet_layer(inputs=x,
                                num_filters=num_filters_out,
                                kernel_size=1,
                                strides=strides,
                                activation=None,
                                batch_normalization=False)
            # and add every reputation
            x = keras.layers.add([x, y])
        
        num_filters_in = num_filters_out
    
    # Add classifier on top
    # v2 has BN_ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                   activation='softmax',
                   kernel_initializer='he_normal')(y)
    
    # Instantiate model
    model = Model(inputs=inputs, outputs=outputs)
    
    return model

In [7]:
import keras.backend as K
K.clear_session()

n = 6
version = 2
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2
    
input_shape = [48,48,1]

model = resnet_v2(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
             optimizer=Adam(learning_rate=lr_schedule(0)),
             metrics=['acc'])
model.summary()


Learning rate:  0.001
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 48, 48, 16)   32          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 48, 48, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 48, 48, 16)   0           batch_normalization_1[0][0]      
_____________________________________________________________________

## Train

In [8]:
dataset = pd.read_csv('../Sample_Pic_OD/fer2013.csv')
train = dataset[['emotion','pixels']][:28709]
val = dataset[['emotion','pixels']][28709:]

In [ ]:
filepath = os.path.join('../emotion_detector_models/model_resnet_best.hdf5')

nb_train_samples = train.shape[0]
nb_validation_samples = val.shape[0]

checkpoint = ModelCheckpoint(filepath=filepath,
                            monitor='val_acc',
                            verbose=1,
                            save_best_only=True)
lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_scheduler, lr_reducer]

model_info = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples // batch_size,
            epochs=epochs,
            callbacks = callbacks,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples // batch_size)


Epoch 1/200
Learning rate:  0.001
897/897 [==============================] - 196s 219ms/step - loss: 2.6561 - acc: 0.2708 - val_loss: 2.2533 - val_acc: 0.3221

Epoch 00001: val_acc improved from -inf to 0.32213, saving model to ../emotion_detector_models/model_resnet_best.hdf5
Epoch 2/200
Learning rate:  0.001
897/897 [==============================] - 178s 199ms/step - loss: 2.0388 - acc: 0.3300 - val_loss: 1.8546 - val_acc: 0.3784

Epoch 00002: val_acc improved from 0.32213 to 0.37839, saving model to ../emotion_detector_models/model_resnet_best.hdf5
Epoch 3/200
Learning rate:  0.001
897/897 [==============================] - 178s 198ms/step - loss: 1.8328 - acc: 0.3606 - val_loss: 1.7412 - val_acc: 0.4092

Epoch 00003: val_acc improved from 0.37839 to 0.40918, saving model to ../emotion_detector_models/model_resnet_best.hdf5
Epoch 4/200
Learning rate:  0.001
897/897 [==============================] - 178s 198ms/step - loss: 1.7560 - acc: 0.3732 - val_loss: 1.6695 - val_acc: 0.4134


## Restore and Retrain

In [ ]:
model.load_weights('../emotion_detector_models/model_resnet_best.hdf5')

epochs=100

filepath = os.path.join('../emotion_detector_models/model_resnet_best.hdf5')

nb_train_samples = train.shape[0]
nb_validation_samples = val.shape[0]

checkpoint = ModelCheckpoint(filepath=filepath,
                            monitor='val_acc',
                            verbose=1,
                            save_best_only=True)
lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_scheduler, lr_reducer]

model_info = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples // batch_size,
            epochs=epochs,
            callbacks = callbacks,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples // batch_size)


Epoch 1/100
Learning rate:  0.001
897/897 [==============================] - 193s 215ms/step - loss: 1.3000 - acc: 0.5278 - val_loss: 1.3391 - val_acc: 0.5181

Epoch 00001: val_acc improved from -inf to 0.51814, saving model to ../emotion_detector_models/model_resnet_best.hdf5
Epoch 2/100
Learning rate:  0.001
897/897 [==============================] - 178s 198ms/step - loss: 1.2999 - acc: 0.5284 - val_loss: 1.6360 - val_acc: 0.5199

Epoch 00002: val_acc improved from 0.51814 to 0.51987, saving model to ../emotion_detector_models/model_resnet_best.hdf5
Epoch 3/100
Learning rate:  0.001
897/897 [==============================] - 178s 198ms/step - loss: 1.2977 - acc: 0.5266 - val_loss: 1.5438 - val_acc: 0.5280

Epoch 00003: val_acc improved from 0.51987 to 0.52799, saving model to ../emotion_detector_models/model_resnet_best.hdf5
Epoch 4/100
Learning rate:  0.001
897/897 [==============================] - 178s 198ms/step - loss: 1.2955 - acc: 0.5301 - val_loss: 1.7732 - val_acc: 0.5250



Epoch 00036: val_acc did not improve from 0.54898
Epoch 37/100
Learning rate:  0.001
897/897 [==============================] - 180s 201ms/step - loss: 1.2516 - acc: 0.5459 - val_loss: 1.6575 - val_acc: 0.5526

Epoch 00037: val_acc improved from 0.54898 to 0.55262, saving model to ../emotion_detector_models/model_resnet_best.hdf5
Epoch 38/100
Learning rate:  0.001
897/897 [==============================] - 180s 200ms/step - loss: 1.2569 - acc: 0.5444 - val_loss: 1.2001 - val_acc: 0.5554

Epoch 00038: val_acc improved from 0.55262 to 0.55542, saving model to ../emotion_detector_models/model_resnet_best.hdf5
Epoch 39/100
Learning rate:  0.001
897/897 [==============================] - 180s 201ms/step - loss: 1.2515 - acc: 0.5449 - val_loss: 1.1212 - val_acc: 0.5399

Epoch 00039: val_acc did not improve from 0.55542
Epoch 40/100
Learning rate:  0.001
897/897 [==============================] - 180s 201ms/step - loss: 1.2466 - acc: 0.5484 - val_loss: 1.0311 - val_acc: 0.5395

Epoch 00040: 

897/897 [==============================] - 180s 201ms/step - loss: 1.2189 - acc: 0.5600 - val_loss: 1.1187 - val_acc: 0.5406

Epoch 00075: val_acc did not improve from 0.55542
Epoch 76/100
Learning rate:  0.001
897/897 [==============================] - 180s 201ms/step - loss: 1.2256 - acc: 0.5574 - val_loss: 1.5972 - val_acc: 0.5585

Epoch 00076: val_acc improved from 0.55542 to 0.55849, saving model to ../emotion_detector_models/model_resnet_best.hdf5
Epoch 77/100
Learning rate:  0.001
897/897 [==============================] - 182s 203ms/step - loss: 1.2262 - acc: 0.5586 - val_loss: 1.4260 - val_acc: 0.5607

Epoch 00077: val_acc improved from 0.55849 to 0.56073, saving model to ../emotion_detector_models/model_resnet_best.hdf5
Epoch 78/100
Learning rate:  0.001
721/897 [=======================>......] - ETA: 32s - loss: 1.2230 - acc: 0.5586

## Restore & Retrain 2

In [ ]:
model.load_weights('../emotion_detector_models/model_resnet_best_r2.hdf5')

epochs=100

filepath = os.path.join('../emotion_detector_models/model_resnet_best_r2.hdf5')

nb_train_samples = train.shape[0]
nb_validation_samples = val.shape[0]

checkpoint = ModelCheckpoint(filepath=filepath,
                            monitor='val_acc',
                            verbose=1,
                            save_best_only=True)
lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=2,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_scheduler, lr_reducer]

model_info = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples // batch_size,
            epochs=epochs,
            callbacks = callbacks,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples // batch_size)


Epoch 1/100
Learning rate:  0.001
897/897 [==============================] - 202s 225ms/step - loss: 1.2113 - acc: 0.5623 - val_loss: 1.5256 - val_acc: 0.5467

Epoch 00001: val_acc improved from -inf to 0.54674, saving model to ../emotion_detector_models/model_resnet_best_r2.hdf5
Epoch 2/100
Learning rate:  0.001
897/897 [==============================] - 182s 203ms/step - loss: 1.2149 - acc: 0.5616 - val_loss: 1.3965 - val_acc: 0.5402

Epoch 00002: val_acc did not improve from 0.54674
Epoch 3/100
Learning rate:  0.001
897/897 [==============================] - 182s 203ms/step - loss: 1.2111 - acc: 0.5600 - val_loss: 1.5045 - val_acc: 0.5570

Epoch 00003: val_acc improved from 0.54674 to 0.55695, saving model to ../emotion_detector_models/model_resnet_best_r2.hdf5
Epoch 4/100
Learning rate:  0.001
897/897 [==============================] - 185s 207ms/step - loss: 1.2147 - acc: 0.5613 - val_loss: 1.5730 - val_acc: 0.5350

Epoch 00004: val_acc did not improve from 0.55695
Epoch 5/100
Le

In [ ]:
print(model_info.history.keys())

import matplotlib.pyplot as plt
plt.plot(model_info.history['loss'])
plt.plot(model_info.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()